# 환경 변수 로드

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# 3-7

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# ChatOpenAI 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", """
     당신은 친절한 한국어 AI 어시스턴트입니다. 
     이전 대화 내용을 고려하여 사용자의 질문에 답변해주세요.
     """),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])


# 가상의 대화 히스토리 생성
history = [
    HumanMessage(content="안녕하세요! 제 이름은 판다스입니다."),
    AIMessage(content="안녕하세요, 판다스님! 어떤 도움이 필요하신가요?"),
    HumanMessage(content="오늘 날씨에 대해 알려주세요."),
    AIMessage(content="네, 오늘 날씨는 맑고 따뜻합니다.")
]

# 프롬프트 템플릿 포맷팅
formatted_prompt = prompt.format(
    history=history,
    input="제 이름이 뭐였죠?"
)

# 포맷팅된 프롬프트 출력
print(formatted_prompt)

System: 
     당신은 친절한 한국어 AI 어시스턴트입니다. 
     이전 대화 내용을 고려하여 사용자의 질문에 답변해주세요.
     
Human: 안녕하세요! 제 이름은 판다스입니다.
AI: 안녕하세요, 판다스님! 어떤 도움이 필요하신가요?
Human: 오늘 날씨에 대해 알려주세요.
AI: 네, 오늘 날씨는 맑고 따뜻합니다.
Human: 제 이름이 뭐였죠?


# 3-8

In [10]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


# 체인 생성
chain = prompt | model

# 메시지 히스토리를 저장할 딕셔너리
store = {}

# 세션 히스토리를 가져오는 함수
def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 메시지 히스토리를 포함한 실행 가능한 체인 생성
chain_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 메모리 체인 객체를 출력 
chain_with_memory

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  history: RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
}), kwargs={}, config={'run_name': 'insert_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function get_session_history at 0x116b6e3e0>, input_messages_key='input', history_messages_key='history', history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

# 3-9

In [11]:
# 대화 실행
config = {"configurable": {"session_id": "user_001"}}

response1 = chain_with_memory.invoke(
    {"input": "안녕하세요! 제 이름은 판다스입니다."},
    config=config,
)

print("AI:", response1.content)

response2 = chain_with_memory.invoke(
    {"input": "제 이름이 뭐였죠?"},
    config=config,
)

print("AI:", response2.content)

AI: 안녕하세요, 판다스님! 반갑습니다. 어떻게 도와드릴까요?
AI: 판다스님이라고 하셨습니다! 맞나요?


# 3-10

In [12]:
# 새로운 세션으로 대화 시작
config_new = {"configurable": {"session_id": "user_002"}}

response3 = chain_with_memory.invoke(
    {"input": "제 이름이 뭐였죠?"},
    config=config_new,
)

print("AI:", response3.content)

AI: 죄송하지만, 이전 대화 내용을 기억할 수 없어서 사용자의 이름을 알 수 없습니다. 다시 말씀해 주시면 좋겠습니다!


# 3-11

In [13]:
# 다시 첫 번째 세션으로 대화를 이어서 시도 
config = {"configurable": {"session_id": "user_001"}}

response4 = chain_with_memory.invoke(
    {"input": "제 이름이 뭐였죠?"},
      config=config,
      )

# 답변 출력 
print("AI:", response4.content)

AI: 판다스님이라고 하셨습니다. 맞나요? 다른 질문이 있으시면 언제든지 말씀해 주세요!


# 3-12

In [14]:
import gradio as gr

# 사용자 메시지를 처리하고 AI 응답을 생성하는 함수
def answer_invoke(message, history):
    # Gradio의 history를 LangChain의 메시지 형식으로 변환
    langchain_history = [
        HumanMessage(content=h[0]) if i % 2 == 0 else AIMessage(content=h[0])
        for i, h in enumerate(history)
    ]
    
    # chain_with_memory 실행
    config = {"configurable": {"session_id": "gradio_user"}}
    response = chain_with_memory.invoke(
        {"input": message, "history": langchain_history},
        config=config
    )
    
    # 생성된 응답 반환
    return response.content

# Gradio ChatInterface 객체 생성
demo = gr.ChatInterface(
    fn=answer_invoke,
    title="한국어 AI 어시스턴트",
    description="이전 대화 내용을 기억하는 AI 어시스턴트입니다.",
    examples=["안녕하세요!", "제 이름은 판다스입니다.", "제 이름이 뭐였죠?"],
    retry_btn="다시 생성",
    undo_btn="되돌리기",
    clear_btn="대화 지우기"
)

# Gradio 인터페이스 실행
demo.launch()

/Users/steve2/Library/Caches/pypoetry/virtualenvs/langchain-book-AlsLlHcI-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [16]:
# Gradio 인터페이스 종료 
demo.close ()

Closing server running on port: 7860
